# Material impacts (workpiece, powder and ingot)

This notebook does Monte Carlo comparative LCA calculations, for GWP only. 

It calculates the GWP of the Ti6-6Al-4V workpiece, Ti6-6Al-4V powder, cold-rolled stainless steel and CoCr ingot at the point where they are on the point to be loaded into the SLM, used for CNC machining or casted.
The results are saved to `results/metal_gwp.csv`.

In [1]:
import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import presamples as ps 
import numpy as np

In [2]:
# Generate objects for analysis

bw.projects.set_current("default")

db_UKR = bw.Database('UKR')
db_AMHTO_jig_steel = bw.Database('AM HTO- jig steel')
db_AMHTO = bw.Database('AM HTO')
db_CMHTO = bw.Database('CM HTO')
cutoff38 = bw.Database('cutoff38')
electricity = bw.Database('Electricity')
anesthesia = bw.Database('Anesthesia')
material = bw.Database('Raw materials')
instruments = bw.Database('Instrumentation')
packaging = bw.Database('Packaging')

# This contain the whole life-cycle impact of the AM and CM HTO
UKR = db_UKR.get('663ac4ef3d314e739f29ec63ea2ca399')
CMHTO = db_CMHTO.get("2f3acaae6c4e4035b2d24c71725b17d8")
AMHTO = db_AMHTO.get("1a637b9baee74199b8027ffeb333279c") 
AMHTO_jig_steel = db_AMHTO_jig_steel.get("1a637b9baee74199b8027ffeb333279c") 
polishing = db_AMHTO.get("8fc93e48b98c4f7487ff73fef8362399_copy1")
anesthesia_gases = anesthesia.get("a00fcd60e79b42bd9f4868e509375673")

AMplate = db_AMHTO.get("51c97fc39fac4b0894237daa8cbd4781_copy1")
CMplate = db_CMHTO.get("5ea661dc6d384105afcf94cc88fb19f6_copy2")


methods = [('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)')]

In [3]:
material.get("5153107d093942aa988fb2497e76ea87_copy1")

'1- Ti6Al4V production' (kilogram, GLO, None)

In [4]:
CoCr =  material.get("08914bd137d643eb86c15648a647e32e_copy1")
Ti6Al4V_workpiece = material.get("ca1c1d7535114301918a4870a775bc76")
Ti6Al4V_powder = material.get("9f5be7543bec47cab0eefbe8257b9291_copy1")
stainless_steel = material.get("809c09c13b0e7129a79aabed4e9e2c08_copy1")


In [5]:
from bw_helpers import sample_comparative_contribution, MyMonteCarloLCA, collect_contribution_samples
from final_activities import final_activities, component_order

In [6]:
all_demands = {
    "CoCr": CoCr,
    "Ti6Al4V_workpiece": Ti6Al4V_workpiece,
    "Ti6Al4V_powder": Ti6Al4V_powder,
    "stainless_steel": stainless_steel
}


In [7]:
%%time
data = []
materials = []
for method in methods:
    for label, demand_dict in all_demands.items():
        lca = bw.MonteCarloLCA({demand_dict: 1}, method)
        results = np.array([next(lca) for _ in range(1000)])

        data.extend(results)
        materials.extend([label] * 1000)  
        df = pd.DataFrame({
            'material': materials,
            'score': data
        })

df

CPU times: total: 12min 1s
Wall time: 11min 15s


,material,score
0,CoCr,31.146573
1,CoCr,30.738272
2,CoCr,32.550213
3,CoCr,31.936240
4,CoCr,32.701381
...,...,...
3995,stainless_steel,7.114017
3996,stainless_steel,5.689740
3997,stainless_steel,7.732278
3998,stainless_steel,4.434877


In [11]:
df.to_csv("results/metal_gwp.csv", index=False)